In [1]:
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [ ]:
#the final df is the result of the merge of the 3 dataframes
# Time:from 2017~now
# 10 coutries:'Brazil', 'Bulgaria', 'Canada', 'China', 'France', 'Germany','Japan', 'Portugal', 'Romania', 'Türkiye'
# 5 seperate variables

## 1\International Trade Data--Monthly

In [69]:
# read csv file
# find import&export data are identical, so just load one of them
df1 = pd.read_csv(r'D:\UN\data_in_use\trade\2017-2021monthly\goods_import.csv', encoding='iso8859-1')

df3 = pd.read_csv(r'D:\UN\data_in_use\trade\2017-2021monthly\service_import.csv', encoding='iso8859-1')

d:\subtle tools\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [22]:
# 处理df1: merchandise import/export
## 只有进出口不一样。Product/Service全都是total
df1 = df1[['Reporting Economy', 'Indicator', 'Year', 'Period', 'Value']]

temp_ex = df1[df1['Indicator'] == 'Total merchandise exports - monthly']
temp_im = df1[df1['Indicator'] == 'Total merchandise imports - monthly']
# merge
temp = pd.merge(temp_ex, temp_im, on=['Reporting Economy', 'Year', 'Period'])
# remove the indicator column
temp = temp[['Reporting Economy', 'Year', 'Period', 'Value_x', 'Value_y']]
# rename the columns
temp.columns = ['Reporting Economy', 'Year', 'Period', 'goods_export', 'goods_import']

merchandise_trade = temp

In [23]:
# select the columns& temp_ex['Reporting Economy']=='China'
# temp_ex[(temp_ex['Year']==2017)& (temp_ex['Period']=='January')& (temp_ex['Reporting Economy']=='China')]
# it's more quick to check the data in excel

In [71]:
# 处理df3/df4: service import/export
# 除了进出口不一样，还有Product/Sector不一样
df3 = df3[['Reporting Economy', 'Indicator', 'Year', 'Period', 'Value','Product/Sector']]
df3 = df3[df3['Product/Sector'] == 'Commercial services']

temp_ex = df3[df3['Indicator'] == 'Commercial services exports of selected economies ?monthly']
temp_im = df3[df3['Indicator'] == 'Commercial services imports of selected economies ?monthly']
# merge
temp = pd.merge(temp_ex, temp_im, on=['Reporting Economy', 'Year', 'Period'])
# remove the indicator column
temp = temp[['Reporting Economy', 'Year', 'Period', 'Value_x', 'Value_y']]
# rename the columns
temp.columns = ['Reporting Economy', 'Year', 'Period', 'service_export', 'service_import']

service_trade = temp

In [72]:
# count the number of countries
len(merchandise_trade['Reporting Economy'].unique())
len(service_trade['Reporting Economy'].unique())

# intersection: full data in such countries
len(set(merchandise_trade['Reporting Economy'].unique()) & set(service_trade['Reporting Economy'].unique()))
set(merchandise_trade['Reporting Economy'].unique()) & set(service_trade['Reporting Economy'].unique())

75

18

15

{'Brazil',
 'Bulgaria',
 'Canada',
 'China',
 'France',
 'Germany',
 'Japan',
 'Korea, Republic of',
 'North Macedonia',
 'Philippines',
 'Portugal',
 'Romania',
 'Türkiye',
 'Ukraine',
 'United States of America'}

In [73]:
# we will remove the uk, because of war
# also, it dose not have the GDP data

## 2\Data of Production Integration Index--Quarterly

In [74]:
gdp = pd.read_csv(r'D:\UN\data_in_use\trade\quarterly\GDP\GDP.csv')

# artificially match the location with the country name(Reporting Economy)
# Bra BGR CAN CHN FRA DEU JPN KOR MKD PHL PRT ROU TUR UKR USA
# select above countries
gdp = gdp[gdp['LOCATION'].isin(['BRA', 'BGR', 'CAN', 'CHN', 'FRA', 'DEU', 'JPN', 'KOR', 'MKD', 'PHL', 'PRT', 'ROU', 'TUR', 'UKR', 'USA'])]

gdp = gdp[['LOCATION', 'TIME', 'Value']]
# change the location to country name
gdp['LOCATION'] = gdp['LOCATION'].replace(['BRA', 'BGR', 'CAN', 'CHN', 'FRA', 'DEU', 'JPN', 'KOR', 'MKD', 'PHL', 'PRT', 'ROU', 'TUR', 'UKR', 'USA'], ['Brazil', 'Bulgaria', 'Canada', 'China', 'France', 'Germany', 'Japan', 'Korea', 'North Macedonia', 'Philippines', 'Portugal', 'Romania', 'Türkiye', 'Ukraine', 'United States'])

In [75]:
# TIME convert to year and month
gdp['TIME'] = gdp['TIME'].astype(str)
gdp['year'] = gdp['TIME'].str[:4]
gdp['month'] = gdp['TIME'].str[4:]
gdp['month'] = gdp['month'].replace(['-Q1', '-Q2', '-Q3', '-Q4'], ['January', 'April', 'July', 'October'])
gdp = gdp[['LOCATION', 'year', 'month', 'Value']]
gdp.columns = ['Reporting Economy', 'Year', 'Period', 'GDP']


In [76]:
# make a difference between the country name and the location
# elementwise comparison
# MKD,PHL,UKR
# finally 12 countries
gdp['Reporting Economy'].unique()

array(['Korea', 'Japan', 'Canada', 'Brazil', 'United States', 'Türkiye',
       'Germany', 'Portugal', 'China', 'France', 'Bulgaria', 'Romania'],
      dtype=object)

In [77]:
invest = pd.read_csv(r'D:\UN\data_in_use\trade\quarterly\investment\investment.csv')

invest = invest[invest['LOCATION'].isin(['BRA', 'BGR', 'CAN', 'CHN', 'FRA', 'DEU', 'JPN', 'KOR', 'MKD', 'PHL', 'PRT', 'ROU', 'TUR', 'UKR', 'USA'])]

In [78]:
invest.LOCATION.unique()
# no china data

array(['CAN', 'FRA', 'DEU', 'JPN', 'KOR', 'PRT', 'TUR', 'USA', 'BRA',
       'BGR', 'ROU'], dtype=object)

## Merge Y

In [79]:
# availabe data
# service_trade; merchandise_trade; gdp; invest

In [80]:
# merge service_trade and merchandise_trade
global_trade = pd.merge(service_trade, merchandise_trade, on=['Reporting Economy', 'Year', 'Period'],how='inner')

In [81]:
len(global_trade['Reporting Economy'].unique())

15

In [82]:
# unique
global_trade['Reporting Economy'].unique()
gdp['Reporting Economy'].unique()
# difference
set(global_trade['Reporting Economy'].unique()) - set(gdp['Reporting Economy'].unique())

array(['Brazil', 'Bulgaria', 'Canada', 'China', 'France', 'Germany',
       'Japan', 'Korea, Republic of', 'Philippines', 'Portugal',
       'Romania', 'Türkiye', 'Ukraine', 'North Macedonia',
       'United States of America'], dtype=object)

array(['Korea', 'Japan', 'Canada', 'Brazil', 'United States', 'Türkiye',
       'Germany', 'Portugal', 'China', 'France', 'Bulgaria', 'Romania'],
      dtype=object)

{'Korea, Republic of',
 'North Macedonia',
 'Philippines',
 'Ukraine',
 'United States of America'}

In [83]:
# remove the countries
global_trade = global_trade[global_trade['Reporting Economy'].isin(['Korea', 'Japan', 'Canada', 'Brazil', 'United States', 'Türkiye',
       'Germany', 'Portugal', 'China', 'France', 'Bulgaria', 'Romania'])]

In [84]:
# unique
# 最后只剩10个
global_trade['Reporting Economy'].unique()

array(['Brazil', 'Bulgaria', 'Canada', 'China', 'France', 'Germany',
       'Japan', 'Portugal', 'Romania', 'Türkiye'], dtype=object)

In [85]:
# merge global_trade and gdp
# ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat
# change the type of year
global_trade['Year'] = global_trade['Year'].astype(str)
full_df = pd.merge(global_trade, gdp, on=['Reporting Economy', 'Year', 'Period'],how='left')

In [86]:
# convert period to number
full_df['Period'] = full_df['Period'].replace(['January', 'February','March','April', 'May','June','July','August','September', 'October','November','December'], 
[1, 2, 3, 4,5,6,7,8,9,10,11,12])
# sort by reporting economy, year and month
full_df = full_df.sort_values(by=['Reporting Economy','Year','Period'], ascending=True)


In [87]:
# foward fill the gdp
full_df['GDP'] = full_df['GDP'].fillna(method='ffill')

In [89]:
# save the data
full_df.to_csv(r'D:\UN\data_in_use\full_df.csv', index=False)